In [2]:
__author__ = 'kent'

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

def weight_varible(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
print("Download Done!")

sess = tf.InteractiveSession()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Download Done!


In [3]:
# paras
W_conv1 = weight_varible([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# conv layer-1
x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1, 28, 28, 1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

* 測試 Layer 1 的輸出

In [6]:
sess.run(tf.initialize_all_variables())
sess.run(h_pool1,
         feed_dict={x:mnist.train.images[:10]}).shape

(10, 14, 14, 32)

In [7]:
# conv layer-2
W_conv2 = weight_varible([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

* 測試 Layer 2 的輸出

In [12]:
sess.run(tf.initialize_all_variables())
sess.run(h_pool2,
         feed_dict={x:mnist.train.images[:10]}).shape

(10, 7, 7, 64)

In [18]:
# full connection
W_fc1 = weight_varible([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

* 測試 Layer 3 的輸出

In [20]:
sess.run(tf.initialize_all_variables())
print sess.run(h_fc1,
         feed_dict={x:mnist.train.images[:10]}).shape

print sess.run(h_fc1_drop,
         feed_dict={x:mnist.train.images[:10],keep_prob:0.75  }).shape



(10, 1024)
(10, 1024)


In [23]:
# output layer: softmax
W_fc2 = weight_varible([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
y_ = tf.placeholder(tf.float32, [None, 10])

In [29]:
sess.run(tf.initialize_all_variables())
print sess.run(y_conv,
         feed_dict={x:mnist.train.images[:20],keep_prob:0.75 }).shape

(20, 10)


In [39]:
# model training
cross_entropy = -tf.reduce_mean(y_ * tf.log(y_conv))

* 測試輸出 Cross Entropy

In [40]:
sess.run(tf.initialize_all_variables())
print sess.run(cross_entropy,
         feed_dict={x:mnist.train.images[:20],keep_prob:0.75,y_ :mnist.train.labels[:20] })

0.979217


In [41]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.arg_max(y_conv, 1), tf.arg_max(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 開始進行 Training

In [ ]:
sess.run(tf.initialize_all_variables())

for i in range(20000):
    batch = mnist.train.next_batch(50)

    if i % 100 == 0:
        train_accuacy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuacy))
    train_step.run(feed_dict = {x: batch[0], y_: batch[1], keep_prob: 0.5})



# 拿測試資料來驗證

In [43]:
# accuacy on test
print("test accuracy %g"%(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})))

test accuracy 0.9741
